In [3]:
import pandas as pd
df_demand  = pd.read_excel('https://github.com/reaniumpy/python/blob/main/INPUT_supply_planning.xlsx?raw=true', 
                   sheet_name = 'Demand', usecols ='A,C,D')
df_demand.head(2)

,Part_No,Job Qty,Job start date
0,BANH-XE,2442,2022-04-11
1,BANH-XE,880,2022-05-03


In [4]:
df_supply  = pd.read_excel('https://github.com/reaniumpy/python/blob/main/INPUT_supply_planning.xlsx?raw=true', 
                   sheet_name = 'Supply', skiprows = 1)
df_supply

,Part_No,Stock on hand,2022-03-28 00:00:00,2022-04-04 00:00:00,2022-04-11 00:00:00,2022-04-18 00:00:00,2022-04-25 00:00:00
0,BANH-XE,54,2388,4776.0,2393.0,4781.0,2398.0
1,CHUONG,0,408,816.0,413.0,821.0,418.0
2,BAN-DAP,0,2200,NaN,NaN,NaN,NaN
3,YEN-XE,41,1959,3918.0,1964.0,3923.0,1969.0


# BƯỚC 2: CUMSUM DEMAND

In [5]:
df_demand_sorted =  df_demand.sort_values(by=['Part_No','Job start date'])
df_demand_sorted['cumsum_demand']=df_demand_sorted.groupby('Part_No')['Job Qty'].cumsum()
df_demand_sorted


,Part_No,Job Qty,Job start date,cumsum_demand
4,BAN-DAP,2200,2022-04-21,2200
5,BAN-DAP,2500,2022-05-01,4700
0,BANH-XE,2442,2022-04-11,2442
1,BANH-XE,880,2022-05-03,3322
2,CHUONG,408,2022-04-10,408
3,CHUONG,1292,2022-04-11,1700
6,YEN-XE,2000,2022-04-13,2000
7,YEN-XE,1542,2022-04-17,3542
8,YEN-XE,248,2022-04-18,3790


# Bước 3: Unpivot dữ liệu Supply

In [6]:

df_supply = df_supply.rename(columns={'Stock on hand':pd.to_datetime('2000-01-01')})
df_supply_melted = df_supply.melt('Part_No').rename(columns={'variable':'supply_date','value':'supply_quanlity'}).sort_values(by=['Part_No','supply_date'])
df_supply_melted['cumsum_supply']= df_supply_melted.groupby('Part_No')['supply_quanlity'].cumsum()
df_supply_melted



,Part_No,supply_date,supply_quanlity,cumsum_supply
2,BAN-DAP,2000-01-01,0.0,0.0
6,BAN-DAP,2022-03-28,2200.0,2200.0
10,BAN-DAP,2022-04-04,NaN,NaN
14,BAN-DAP,2022-04-11,NaN,NaN
18,BAN-DAP,2022-04-18,NaN,NaN
22,BAN-DAP,2022-04-25,NaN,NaN
0,BANH-XE,2000-01-01,54.0,54.0
4,BANH-XE,2022-03-28,2388.0,2442.0
8,BANH-XE,2022-04-04,4776.0,7218.0
12,BANH-XE,2022-04-11,2393.0,9611.0


# Bước 4 Join Demand với Supply

In [7]:
df_full = pd.merge(df_demand_sorted,df_supply_melted, how = 'outer', on=['Part_No'])
#pd.merge(left, right, how='inner', on=['key1', 'key2'])
df_full

,Part_No,Job Qty,Job start date,cumsum_demand,supply_date,supply_quanlity,cumsum_supply
0,BAN-DAP,2200,2022-04-21,2200,2000-01-01,0.0,0.0
1,BAN-DAP,2200,2022-04-21,2200,2022-03-28,2200.0,2200.0
2,BAN-DAP,2200,2022-04-21,2200,2022-04-04,NaN,NaN
3,BAN-DAP,2200,2022-04-21,2200,2022-04-11,NaN,NaN
4,BAN-DAP,2200,2022-04-21,2200,2022-04-18,NaN,NaN
5,BAN-DAP,2200,2022-04-21,2200,2022-04-25,NaN,NaN
6,BAN-DAP,2500,2022-05-01,4700,2000-01-01,0.0,0.0
7,BAN-DAP,2500,2022-05-01,4700,2022-03-28,2200.0,2200.0
8,BAN-DAP,2500,2022-05-01,4700,2022-04-04,NaN,NaN
9,BAN-DAP,2500,2022-05-01,4700,2022-04-11,NaN,NaN


In [22]:
df_enough = df_full[df_full['cumsum_demand'] <= df_full['cumsum_supply']]
df_first_supply = df_enough.groupby(['Part_No','cumsum_demand']).first().reset_index()
df_first_supply

,Part_No,cumsum_demand,Job Qty,Job start date,supply_date,supply_quanlity,cumsum_supply
0,BAN-DAP,2200,2200,2022-04-21,2022-03-28,2200.0,2200.0
1,BANH-XE,2442,2442,2022-04-11,2022-03-28,2388.0,2442.0
2,BANH-XE,3322,880,2022-05-03,2022-04-04,4776.0,7218.0
3,CHUONG,408,408,2022-04-10,2022-03-28,408.0,408.0
4,CHUONG,1700,1292,2022-04-11,2022-04-18,821.0,2458.0
5,YEN-XE,2000,2000,2022-04-13,2022-03-28,1959.0,2000.0
6,YEN-XE,3542,1542,2022-04-17,2022-04-04,3918.0,5918.0
7,YEN-XE,3790,248,2022-04-18,2022-04-04,3918.0,5918.0


In [23]:
df_final = df_first_supply.merge(df_demand,how='right')
df_final

,Part_No,cumsum_demand,Job Qty,Job start date,supply_date,supply_quanlity,cumsum_supply
0,BANH-XE,2442.0,2442,2022-04-11,2022-03-28,2388.0,2442.0
1,BANH-XE,3322.0,880,2022-05-03,2022-04-04,4776.0,7218.0
2,CHUONG,408.0,408,2022-04-10,2022-03-28,408.0,408.0
3,CHUONG,1700.0,1292,2022-04-11,2022-04-18,821.0,2458.0
4,BAN-DAP,2200.0,2200,2022-04-21,2022-03-28,2200.0,2200.0
5,BAN-DAP,NaN,2500,2022-05-01,NaT,NaN,NaN
6,YEN-XE,2000.0,2000,2022-04-13,2022-03-28,1959.0,2000.0
7,YEN-XE,3542.0,1542,2022-04-17,2022-04-04,3918.0,5918.0
8,YEN-XE,3790.0,248,2022-04-18,2022-04-04,3918.0,5918.0


# Bước 5 Calculate the number of late day

In [25]:
df_final['late_date'] = df_final['supply_date'] - df_final['Job start date']
df_final

,Part_No,cumsum_demand,Job Qty,Job start date,supply_date,supply_quanlity,cumsum_supply,late_date
0,BANH-XE,2442.0,2442,2022-04-11,2022-03-28,2388.0,2442.0,-14 days
1,BANH-XE,3322.0,880,2022-05-03,2022-04-04,4776.0,7218.0,-29 days
2,CHUONG,408.0,408,2022-04-10,2022-03-28,408.0,408.0,-13 days
3,CHUONG,1700.0,1292,2022-04-11,2022-04-18,821.0,2458.0,7 days
4,BAN-DAP,2200.0,2200,2022-04-21,2022-03-28,2200.0,2200.0,-24 days
5,BAN-DAP,NaN,2500,2022-05-01,NaT,NaN,NaN,NaT
6,YEN-XE,2000.0,2000,2022-04-13,2022-03-28,1959.0,2000.0,-16 days
7,YEN-XE,3542.0,1542,2022-04-17,2022-04-04,3918.0,5918.0,-13 days
8,YEN-XE,3790.0,248,2022-04-18,2022-04-04,3918.0,5918.0,-14 days
